In [1]:
import torch
import random
import librosa
import numpy as np
from datasets import load_dataset
from IPython.display import Audio
from audiodiffusion import AudioDiffusion
from audiodiffusion.mel import Mel
import torchaudio
import torchvision
import soundfile as sf

ModuleNotFoundError: No module named 'audiodiffusion'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
generator = torch.Generator(device=device)

In [ ]:
model_id = "teticio/audio-diffusion-ddim-256"
audio_diffusion = AudioDiffusion(model_id=model_id)
mel: Mel = audio_diffusion.pipe.mel

In [ ]:
target_sample_rate = 22050

In [ ]:
def readAudioFile(path):
    print("Reading audio file: " + path)

    raw_audio, sample_rate = librosa.load(path, sr=target_sample_rate)
    mel.load_audio(raw_audio=raw_audio)
    image = mel.audio_slice_to_image(0)
    audio=mel.get_audio_slice(0)

    # image, (sample_rate, audio) = audio_diffusion.generate_spectrogram_and_audio_from_audio(source)
    print("Encoding audio file: " + path)
    noise = audio_diffusion.pipe.encode([image])
    return [image, audio, noise, sample_rate]

In [3]:
source = "./bucky.wav"
target = "./makeba.wav"

In [7]:
source_audio = librosa.load(source)
print(source_audio)
source_audio.shape()

(array([8.4040391e-07, 4.5585525e-06, 3.7830046e-06, ..., 2.4618150e-03,
       1.8765979e-03, 2.3382534e-03], dtype=float32), 22050)


AttributeError: 'tuple' object has no attribute 'shape'

In [10]:
source_image, source_audio, source_noise, sample_rate= readAudioFile(source)

Reading audio file: ./bucky.wav
Encoding audio file: ./bucky.wav


  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
taget_image, target_audio, target_noise, _ = readAudioFile(target)

Reading audio file: ./makeba.wav
Encoding audio file: ./makeba.wav


  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
print(len(source_audio))

131071


In [13]:
print(len(target_audio))

131071


In [14]:
sample_rate

22050

In [15]:
print(str(source_image.width) + ":" + str(source_image.height))

256:256


In [16]:
alpha = 0.5  #@param {type:"slider", min:0, max:1, step:0.1}
_, (sample_rate, audio) = audio_diffusion.generate_spectrogram_and_audio(
    noise=audio_diffusion.pipe.slerp(
        torch.tensor(source_noise),
        torch.tensor(target_noise),
        alpha),
    generator=generator)

  0%|          | 0/50 [00:00<?, ?it/s]

In [29]:
display(Audio(source_audio, rate=sample_rate))
display(Audio(target_audio, rate=sample_rate))
display(Audio(audio, rate=sample_rate))

In [34]:
audio

array([ 0.00130178,  0.00151524,  0.00181172, ..., -0.00033109,
       -0.00182112, -0.00240945], dtype=float32)

In [33]:
target_audio.min()

-0.7131028

-1.0

In [39]:
normalized_audio = librosa.util.normalize(audio)
sf.write('./output.wav', normalized_audio, sample_rate, subtype='PCM_24')

print("Done. Saved audio file.")

Done. Saved audio file.
